## Librerias

In [1]:
import pandas as pd
import numpy as np

## Carga de datasets

In [105]:
url = 'https://raw.githubusercontent.com/ronalcabrera/PG_Olist/main/Datasets/olist_customers_dataset.csv'
customers = pd.read_csv(url, delimiter=',', encoding='UTF-8')
customers

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


## Columnas `customer_id` & `customer_unique_id`

Verifico si hay duplicados:

In [29]:
customers[['customer_id','customer_unique_id']].groupby(['customer_id'])['customer_unique_id'].count()

customer_id
00012a2ce6f8dcda20d059ce98491703    1
000161a058600d5901f007fab4c27140    1
0001fd6190edaaf884bcaf3d49edf079    1
0002414f95344307404f0ace7a26f1d5    1
000379cdec625522490c315e70c7a9fb    1
                                   ..
fffecc9f79fd8c764f843e9951b11341    1
fffeda5b6d849fbd39689bb92087f431    1
ffff42319e9b2d713724ae527742af25    1
ffffa3172527f765de70084a7e53aae8    1
ffffe8b65bbe3087b653a978c870db99    1
Name: customer_unique_id, Length: 99441, dtype: int64

Se verifica que hay la misma cantidad de filas luego de agrupar por `customer_id`, no hay customer_id duplicados.<br>
Inconcluso a que se refiere con la columna `customer_unique_id` ya que son todos únicos.

## Columna `customer_zip_code_prefix`

Verifico el tipo de dato este correcto

In [36]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


Por el tipo de dato de la columna `customer_zip_code_prefix` se verifica que son todos números enteros, procedo a listarlos y contarlos.

In [43]:
zip_code_prefix = customers['customer_zip_code_prefix'].unique().tolist()
zip_code_prefix.sort()
len(zip_code_prefix)

14994

Verifico que los prefijos y las abreviaturas esten bien

Primero verifico que los valores mínimos esten dentro del rango, luego verifico los máximos

In [83]:
df = customers[['customer_state','customer_zip_code_prefix']].groupby(['customer_state'], ).min()
df['prefix_max'] = customers[['customer_state','customer_zip_code_prefix']].groupby(['customer_state'], ).max().values
df.sort_values('customer_zip_code_prefix')

,customer_zip_code_prefix,prefix_max
customer_state,,
SP,1003,19970
RJ,20010,28999
ES,29010,29985
MG,30110,39995
BA,40010,48990
SE,49000,49995
PE,50010,56980
AL,57010,57975
PB,58010,58995


Problemas:<br>
• AM y RR comparten rango entre 69301-69380 (RR incluído dentro del rango de AM)<br>
• DF y GO comparten rango entre 72800-73402<br>
Verifico el nombre de los estados en los intervalos para chequear

In [85]:
customers[(customers['customer_zip_code_prefix']>69300) & 
            (customers['customer_zip_code_prefix']<69380) & 
            (customers['customer_state']=='AM')]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state


No se pisan, ahora verifico los que estan fuera del rango de RR y siguien siendo AM

In [87]:
customers[(customers['customer_zip_code_prefix']>69380) & (customers['customer_state']=='AM')]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
5713,6e8a9c87724e18b7e567193cdd3e4b45,8fe805d171f920bf0ae393368aad5316,69800,humaita,AM
56845,df6d93efba53a613bf2365f7945febb0,9484cca8fd716e613bc1a0bb17ed0172,69800,humaita,AM
68437,19faaa8953bbd5166298b6f2a3f84298,3246cc1ffb8b0e719e5ba4db0443185d,69460,coari,AM
76259,a685e039c2149f0278def380b654b0b8,ccb89ab59ba8eec8518110077868630a,69800,humaita,AM
80440,ec9dfbd6b5010c0bb5164e740e002d6d,e084d4891d0ffe9337c6b562b2df42b2,69800,humaita,AM


Los datos son correctos, ante cualquier duda referir:<br>
• humaita https://codigo-postal.org/brasil/am/humaita/<br>
• coari https://codigo-postal.org/brasil/am/coari/

In [96]:
customers[(customers['customer_zip_code_prefix']>72799) &
         (customers['customer_zip_code_prefix']<73403) & 
         (customers['customer_state']=='DF')].groupby(['customer_city'])[['customer_state']].count()

,customer_state
customer_city,
brasilia,173
sobradinho,1


In [97]:
customers[(customers['customer_zip_code_prefix']>72799) &
         (customers['customer_zip_code_prefix']<73403) & 
         (customers['customer_state']=='DF')]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
144,d6ea00d4a2dca6a01c708931b325a9e5,1d4626b197f66aa6129c135ab09f1a7b,73020,brasilia,DF
324,8bbcf9a0717a1cb212b45522ef61972c,3078dfe83b099ae6ebe055be3abe0c0d,73035,brasilia,DF
1024,a55002f0d9ba4aa981a3cc93c669617a,4f463757edb0250ec353ab6f3610cf34,73330,brasilia,DF
1235,7c6c12489e36aafa4ab5dfcfef887475,55133cf96f44c834ac51d0e3b678c7a8,73035,brasilia,DF
2095,baca33004aa726524d5a891853100559,aa67457bb07fde4e952d67b038135bee,73369,brasilia,DF
...,...,...,...,...,...
96886,d43a38a1b87c85d51d3c00ab2dd19390,e906b389ad2bfb4058e80f63e73b3458,73252,brasilia,DF
97721,484572a7bd433021fd437bc6f29c8544,af7bb09e58dadca0e5f11e1a3ad0c2a9,73350,brasilia,DF
97986,5e28718800958c9b17749226df1f8b02,2e7bb3aad177a6e3bf8abd2de27c2a09,73368,brasilia,DF
98371,e62cf9e803aee2389d750a99f8a2986e,e5f6bf6e4999102bc5405eb858869122,73370,brasilia,DF


Los datos son correctos, ante cualquier duda referir:<br>
• Brasilia https://www.worldpostalcodes.org/l1/pt/br/brasil/lista/r1/lista-dos-codigos-postais-em-distrito-federal


In [107]:
customers[(customers['customer_zip_code_prefix']>72799) &
         (customers['customer_zip_code_prefix']<73403) & 
         (customers['customer_city']=='sobradinho')]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
8262,075ef61489ef7122c2a0f69e79c55ff3,757423381d83adecca877fc69cfb8540,73060,sobradinho,DF


Este código postal corresponde a Brasilia, que si se encuentra en Distrito Federal como indica en la siguiente columna,<br>
el correspondiente a `sobradinho` es 48925, asi que se corregirá la ciudad a Brasilia.

In [110]:
customers.iloc[8262,3]='brasilia'
customers.iloc[8262,3]


'brasilia'

## Columna `customer_city`

No vamos a trabajar con esta columna, asi que no se le hará ningún tratamiento

## Columna `customer_state`

In [116]:
print('Cantidad de federaciones:',len(customers.customer_state.unique().tolist()))
print(customers.customer_state.unique().tolist())

Cantidad de federaciones: 27
['SP', 'SC', 'MG', 'PR', 'RJ', 'RS', 'PA', 'GO', 'ES', 'BA', 'MA', 'MS', 'CE', 'DF', 'RN', 'PE', 'MT', 'AM', 'AP', 'AL', 'RO', 'PB', 'TO', 'PI', 'AC', 'SE', 'RR']


Los códigos son correctos, nada que agregar.